# Accessing data from the Materials Project (legacy)

In [1]:
import pprint

from pymatgen.ext.matproj import MPRester

# If you haven't set the API KEY in your pmgrc.yaml file or as an environment variable (PMG_MAPI_KEY)
# Put the key in the MPRester call e.g MPRester("API_KEY")
mpr = MPRester()

## Getting structures

Let's say we want to find all the structures which contained Lithium and had a band gap higher than 1 eV. We can directly query the MP using MongoDB syntax.
To query for a particular element, we use the `elements` parameter. To query for a particular band gap value we use the `band_gap` parameter. The criteria passed to `MPRester` is as follows:
```
criteria = 'elements':{'$all':['Li']}, 'band_gap' = {'$gte':1}

```
- `'$all'` indicates that we want all the materials returned by the query to contain Li
- `'$gte'` indicates that we want all the materials returned by the the query to have a value of the property (band gap) to be ***greater than*** 1

For the parameters that can be used in a Materials Project query, see the documentation (https://github.com/materialsproject/mapidoc/tree/master/materials)

For a full description of the MongoDB query operators, please refer to the MongoDB manual (https://www.mongodb.com/docs/manual/reference/operator/query/). 
Being able to use these operators will enhance your querying.

In [2]:
# This defines the query to the materials project
criteria = {'elements': {'$all':['Li']}, 'band_gap':{'$gte':1}}

# This defines the properties that we are interested in
properties= ['material_id', 'pretty_formula', 'structure' ]

data = mpr.query(criteria=criteria, properties=properties)

print(len(data))

100%|██████████| 8978/8978 [00:58<00:00, 153.44it/s]

8978


In [3]:
# data is list of dictionaries
pprint.pprint(data[0])

{'material_id': 'mp-1004373',
 'pretty_formula': 'LiMnO2',
 'structure': Structure Summary
Lattice
    abc : 2.873243 5.301665 10.086128
 angles : 90.0 90.0 90.0
 volume : 153.64170389541192
      A : 2.873243 0.0 0.0
      B : 0.0 5.301665 0.0
      C : 0.0 0.0 10.086128
PeriodicSite: Li (0.7183, 0.5434, 6.1088) [0.2500, 0.1025, 0.6057]
PeriodicSite: Li (2.1549, 4.7583, 3.9773) [0.7500, 0.8975, 0.3943]
PeriodicSite: Li (0.7183, 2.1074, 1.0657) [0.2500, 0.3975, 0.1057]
PeriodicSite: Li (2.1549, 3.1942, 9.0204) [0.7500, 0.6025, 0.8943]
PeriodicSite: Mn (2.1549, 3.2164, 6.3788) [0.7500, 0.6067, 0.6324]
PeriodicSite: Mn (0.7183, 2.0853, 3.7074) [0.2500, 0.3933, 0.3676]
PeriodicSite: Mn (0.7183, 0.5656, 8.7504) [0.2500, 0.1067, 0.8676]
PeriodicSite: Mn (2.1549, 4.7361, 1.3357) [0.7500, 0.8933, 0.1324]
PeriodicSite: O (0.7183, 2.4056, 7.4410) [0.2500, 0.4537, 0.7377]
PeriodicSite: O (2.1549, 2.8961, 2.6451) [0.7500, 0.5463, 0.2623]
PeriodicSite: O (2.1549, 5.0564, 7.6882) [0.7500, 0.9537, 0

Using some additional MongoDB operators we could refine our query. 
For example, we could filter out radioactivate elements and trainsition metals in our query using the `'$nin'` operator.

In [4]:
# A list of radioactive elements
radioactive_elements=['Tc', 'Pm', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

# A list of transition metal elements excluding Scandium (Sc), Yttrium (Y), Zirconium (Zr) and Niobium (Nb)
transition_metals = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'La', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Ac']

# Merge the lists
not_wanted = radioactive_elements + transition_metals

# Define the query criteria
criteria = {'elements': {'$nin':not_wanted,'$all':['Li'] }, 'band_gap':{'$gte':1}}

# Define the properties we are interested in
properties= ['material_id', 'pretty_formula', 'structure' ]

# Query the MP
data = mpr.query(criteria=criteria, properties=properties)

print(len(data))

100%|██████████| 1932/1932 [00:12<00:00, 155.63it/s]


1932


From the above query, we can see that using the MongoDB query operators can enable intelligent querying of the MP, in very few lines of code.